# DRAFT - Run SQL Workloads - Insert and Update Data - DRAFT

This Jupyter Notebook contains code to run SQL workloads across databases. 

The Db2 Data Management Console is more than a graphical user interface. It is a set of microservices that you can use to build custom applications to automate your use of Db2.

This Jupyter Notebook contains examples of how to use the Open APIs and the composable interface that are available in the Db2 Data Management Console. Everything in the User Interface is also available through an open and fully documented RESTful Services API. The full set of APIs are documented as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to the documentation directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

You can also embed elements of the full user interface into an IFrame by constructing the appropriate URL.

This hands on lab will be calling the Db2 Data Management Console as a service. However you can explore it through the user interface as well. Just click on the following link to try out the console that is already and setup in this lab: http://localhost:11080/console. If you have not already logged in you can use the following:
* Userid: db2inst1
* Password: db2inst1

### Import Helper Classes
For more information on these classes, see the lab: [Automate Db2 with Open Console Services](http://localhost:8888/notebooks/Db2_Data_Management_Console_Overview.ipynb)

In [ ]:
%run ./dmc_setup.ipynb

### Db2 Data Management Console Connection
To connect to the Db2 Data Management Console service you need to provide the URL, the service name (v4) and profile the console user name and password as well as the name of the connection profile used in the console to connect to the database you want to work with. For this lab we are assuming that the following values are used for the connection:
* Userid: db2inst1
* Password: db2inst1
* Connection: sample

**Note:** If the Db2 Data Management Console has not completed initialization, the connection below will fail. Wait for a few moments and then try it again.

In [ ]:
# Connect to the Db2 Data Management Console service
Console  = 'http://localhost:11080'
profile  = 'SAMPLE'
user     = 'DB2INST1'
password = 'db2inst1'

# Set up the required connection
profileURL = "?profile="+profile
databaseAPI = Db2(Console+'/dbapi/v4')

if databaseAPI.authenticate(user, password, profile) :
    print("Token Created")
else : 
    print("Token Creation Failed")
database = Console

### Confirm the connection
To confirm that your connection is working you can check the status of the moitoring service. You can also check your console connection to get the details of the specific database connection you are working with. Since your console user id and password may be limited as to which databases they can access you need to provide the connection profile name to drill down on any detailed information for the database.

In [ ]:
# List Monitoring Profile
r = databaseAPI.getProfile(profile)
json = databaseAPI.getJSON(r)
print(json)

### SQL Scripts Used to Generate Work
We are going to define a scripts that we will use during this lab to insert and update rows. 

In [ ]:
def addEmployees(startingENO, addENO) :
    addMoreRows = \
    'INSERT INTO EMPLOYEES ' \
    'WITH DT(ENO) AS (VALUES('+str(startingENO)+') UNION ALL SELECT ENO+1 FROM DT WHERE ENO < '+str(startingENO+addENO-1)+')' \
    '''
        SELECT ENO,
        RAND() * 500,
        TRANSLATE(CHAR(INTEGER(RAND()+500000)),
        CASE MOD(ENO,5) WHEN 0 THEN 'aeiou' || 'bcdfg'
            WHEN 1 THEN 'aeiou' || 'hjklm'
            WHEN 2 THEN 'aeiou' || 'npqrs'
            WHEN 3 THEN 'fredr' || 'annab'
            ELSE 'aeiou' || 'twxyz' END,
            '1234567890') AS LASTNAME,
       CURRENT DATE - (RAND()*10957) DAYS AS HIREDATE,
       INTEGER(1000*RAND()*200) AS SALARY
       FROM DT;
    UPDATE  DEPARTMENTS SET REVENUE = REVENUE + INT(100*RAND()*50);    
    '''
    return addMoreRows
print("Defined Add More Rows String Routine")


### Creating a Routine to Run an SQL Script
To make things easier we can create reusable routines that will included everything we have developed so far. By running the next two steps, you create two routines that you can call by passing parameters to them. 

While we could create a single routine to run SQL and then display the results, we are creating two different routines so that we can display the results differently later in the lab. 

In [ ]:
def runSQL(profile,user, password, sqlText):
    
    if databaseAPI.authenticate(user, password, profile) :

        # Run the SQL Script and return the runID for later reference 
        runID = databaseAPI.getJSON(databaseAPI.runSQL(sqlText))['id'] 

        # See if there are any results yet for this job
        json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
        
        # If the REST call returns an error return the json with the error to the calling routine
        if 'errors' in json :
            return json
        # Append the results from each statement in the script to the overall combined JSON result set
        fulljson = json

        while json['results'] != [] or (json['status'] != "completed" and json['status'] != "failed") :
            json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))

            # Get the results from each statement as they return and append the results to the full JSON 
            for results in json['results'] :
                fulljson['results'].append(results)
            # Wait 250 ms for more results
            time.sleep(0.25) 
        return fulljson
    else :
        print('Could not authenticate')
print('runSQL routine defined')

For each pass through the script to add employees and update department revenue we first need to determine the current maximum employee number.

In [ ]:
def maxEmployees(profile) :
    json = runSQL(profile, user, password, 'SELECT MAX(ENO) AS COUNT FROM EMPLOYEES')
    return json['results'][0]['rows'][0][0]

profileList = ['SAMPLE','HISTORY']
for profile in profileList :
    print(profile + ": " + maxEmployees(profile))

The next cell loops through a list of databases and runs then repeatedly. You can set the number of new employees to add for each repetition and now many times to repeat the cycle. You can also set a time for how long to wait between updates. 

In [ ]:
profileList = ['SAMPLE','HISTORY']

reps = 50
waitTimeSec = 2
addRows = 100

for x in range(0, reps):
    for profile in profileList :
        json = runSQL(profile, user, password, addEmployees(int(maxEmployees(profile))+1, addRows))
        while 'errors' in json:
            print('    * Trying again *')
            json = runSQL(profile, user, password, addEmployees(int(maxEmployees(profile))+1, addRows))
        print(str(json['results'][0]['rows_affected']) + ' rows added to ' + profile)
    time.sleep(waitTimeSec)

#### Credits: IBM 2019, Peter Kohlmann [kohlmann@ca.ibm.com]